<header>
   <p  style='font-size:36px;font-family:Arial; color:#F0F0F0; background-color: #00233c; padding-left: 20pt; padding-top: 20pt;padding-bottom: 10pt; padding-right: 20pt;'>
       NonLinearCombineFit and NonLinearCombineTransform function in Vantage
  <br>
       <img id="teradata-logo" src="https://storage.googleapis.com/clearscape_analytics_demo_data/DEMO_Logo/teradata.svg" alt="Teradata" style="width: 125px; height: auto; margin-top: 20pt;">
    </p>
</header>

<p style = 'font-size:20px;font-family:Arial;color:#00233C'><b>Introduction</b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The NonLinearCombineFit function returns the target columns and a 
    specified formula which uses the non-linear combination of existing features.
    The NonLinearCombineTransform function generates a new feature
    by taking a non-linear combination of existing features using the
    parameters from the output of NonLinearCombineFit function.
<br> In this notebook we will see how we can use the NonLinearCombineFit and NonLinearCombineTransform functions available in Vantage to create new feature columns in dataset.</p>

<hr style="height:2px;border:none;background-color:#00233C;">
<b style = 'font-size:20px;font-family:Arial;color:#00233C'>1. Initiate a connection to Vantage</b>

In [ ]:
from teradataml import *

# Modify the following to match the specific client environment settings
display.max_rows = 5

<hr style="height:1px;border:none;background-color:#00233C;">
<p style = 'font-size:18px;font-family:Arial;color:#00233c'><b>1.1 Connect to Vantage</b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>You will be prompted to provide the password. Enter your password, press the Enter key, and then use the down arrow to go to the next cell.</p>

In [ ]:
%run -i ../../UseCases/startup.ipynb
eng = create_context(host = 'host.docker.internal', username='demo_user', password = password)
print(eng)

In [ ]:
%%capture
execute_sql('''SET query_band='DEMO=PP_NonLinearCombineFitandTransform_Python.ipynb;' UPDATE FOR SESSION; ''')

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Begin running steps with Shift + Enter keys. </p>

<hr style='height:1px;border:none;background-color:#00233C;'>

<p style = 'font-size:18px;font-family:Arial;color:#00233c'><b>1.2 Getting Data for This Demo</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>We have provided data for this demo on cloud storage. You can either run the demo using foreign tables to access the data without any storage on your environment or download the data to local storage, which may yield faster execution. Still, there could be considerations of available storage. Two statements are in the following cell, and one is commented out. You may switch which mode you choose by changing the comment string.</p>

In [ ]:
%run -i ../../UseCases/run_procedure.py "call get_data('DEMO_TitanicSurvival_cloud');"        # Takes 30 seconds
#%run -i ../../UseCases/run_procedure.py "call get_data('DEMO_TitanicSurvival_local');" 

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Next is an optional step – if you want to see the status of databases/tables created and space used.</p>

In [ ]:
%run -i ../../UseCases/run_procedure.py "call space_report();"        # Takes 10 seconds

<hr style="height:2px;border:none;background-color:#00233C;">
<b style = 'font-size:20px;font-family:Arial;color:#00233C'>2. Data Exploration</b>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Create a "Virtual DataFrame" that points to the data set in Vantage. Check the shape of the dataframe as check the datatype of all the columns of the dataframe.</p>

In [ ]:
tdf = DataFrame(in_schema("DEMO_TitanicSurvival", "Passenger_Data"))
print("Shape of the data: ", tdf.shape)
tdf

In [ ]:
tdf.tdtypes

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>This notebook demonstrates the use of `teradataml` to analyze Titanic passenger data and calculate a derived feature called the `luxury_score`. The `luxury_score` is a non-linear combination of family size, ticket fare, and passenger class, providing insights into passenger profiles.
</p>

In [ ]:
help(NonLinearCombineFit)

In [ ]:
# Fit the formula to calculate luxury_score
fit_result = NonLinearCombineFit(
    data=tdf,
    target_columns=["sibsp", "parch", "fare", "pclass"],
    formula="Y=(X0 + X1 + 1) * (X2 + X3 * 10)",
    result_column="luxury_score"
)

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Use the fitted model to calculate the `luxury_score` for each passenger.
</p>

In [ ]:
help(NonLinearCombineTransform)

In [ ]:
# Transform and add the luxury_score column
transformed_data = NonLinearCombineTransform(
    data=tdf,
    object=fit_result,
    accumulate=["passenger", "passengername", "fare", "pclass", "sibsp", "parch"]
)

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The resulting DataFrame includes the calculated `luxury_score` column.
</p>

In [ ]:
# Print the transformed DataFrame
transformed_data.result

<hr style="height:2px;border:none;background-color:#00233C;">
<b style = 'font-size:20px;font-family:Arial;color:#00233C'>2. Cleanup</b>

<hr style="height:1px;border:none;background-color:#00233C;">
<p style = 'font-size:18px;font-family:Arial;color:#00233C'> <b>Databases and Tables </b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The following code will clean up tables and databases created above.</p>

In [ ]:
%run -i ../../UseCases/run_procedure.py "call remove_data('DEMO_TitanicSurvival');"        # Takes 10 seconds

In [ ]:
remove_context()

<hr style="height:1px;border:none;background-color:#00233C;">
<p style = 'font-size:16px;font-family:Arial;color:#00233C'><b>Links:</b></p>
<ul style = 'font-size:16px;font-family:Arial'>
    <li>Teradataml Python reference: <a href = 'https://docs.teradata.com/search/all?query=Python+Package+User+Guide&content-lang=en-US'>here</a></li>
    <li>NonLinearCombineFit function reference: <a href = 'https://docs.teradata.com/search/all?query=NonLinearCombineFit&value-filters=prodname~%2522Teradata+Package+for+Python%2522*vrm_release~%252220.00.00.03%2522&content-lang=en-US'>here</a></li>
    <li>NonLinearCombineTransform function reference: <a href = 'https://docs.teradata.com/search/all?query=NonLinearCombineTransform&value-filters=prodname~%2522Teradata+Package+for+Python%2522*vrm_release~%252220.00.00.03%2522&content-lang=en-US'>here</a></li>
</ul>

<footer style="padding-bottom:35px; background:#f9f9f9; border-bottom:3px solid #00233C">
    <div style="float:left;margin-top:14px">ClearScape Analytics™</div>
    <div style="float:right;">
        <div style="float:left; margin-top:14px">
            Copyright © Teradata Corporation - 2025. All Rights Reserved
        </div>
    </div>
</footer>